## 任务一：HMM模型用于中文分词

任务一评分标准：
1. 共有8处TODO需要填写，每个TODO计1-2分，共9分，预计代码量30行；
2. 允许自行修改、编写代码完成，对于该情况，请补充注释以便于评分，否则结果不正确将导致较多的扣分；
3. 实验报告(python)/用于说明实验的文字块(jupyter notebook)不额外计分，但不写会导致扣分。

注：本任务仅在短句子上进行效果测试，因此对概率的计算可直接进行连乘。在实践中，常先对概率取对数，将连乘变为加法来计算，以避免出现数值溢出的情况。

导入HMM参数，初始化所需的起始概率矩阵，转移概率矩阵，发射概率矩阵

In [20]:
import pickle
import numpy as np

In [21]:
with open("hmm_parameters.pkl", "rb") as f:
    hmm_parameters = pickle.load(f)

# 非断字（B）为第0行，断字（I）为第1行
# 发射概率矩阵中，词典大小为65536，以汉字的ord作为行key
start_probability = hmm_parameters["start_prob"]  # shape(2,)
trans_matrix = hmm_parameters["trans_mat"]  # shape(2, 2)
emission_matrix = hmm_parameters["emission_mat"]  # shape(2, 65536)

定义待处理的句子

In [22]:
# TODO: 将input_sentence中的xxx替换为你的姓名（1分）
input_sentence = "柳纪宇是一名优秀的学生"


实现viterbi算法，并以此进行中文分词

viterbi算法实现过程：

TODO1:  将第一个位置的最大概率值定义为初始概率和发射概率的乘积。

TODO2:  接着依次更新位置为i（从1到len(sent_ord)-1的整数）的最大概率值矩阵和path矩阵的值。更新位置i状态h的最大概率值（dp[h][i]）的方法为比较上个位置为0和上个位置为1的概率（dp[0][i-1]和d[1][i-1]）与转移到0的转移概率（trans_mat[0][h]和trans_mat[1][h]）的乘积大小，将乘积更大者乘以该位置的发射概率（emission_mat[h][send_ord[i]]）赋给该位置的最大概率值，并将path[h][i]置为概率更大的乘积对应的上个位置的状态值。

TODO3:  以i为索引遍历dp[0][i]与dp[1][i]，比较二者大小，将较大者的状态设置为labels[i]的值。

In [23]:
def viterbi(sent_orig, start_prob, trans_mat, emission_mat):
    """
    viterbi算法进行中文分词

    Args:
        sent_orig: str - 输入的句子
        start_prob: numpy.ndarray - 起始概率矩阵
        trans_mat: numpy.ndarray - 转移概率矩阵
        emission_mat: numpy.ndarray - 发射概率矩阵

    Return:
        str - 中文分词的结果
    """
    
    #  将汉字转为数字表示
    sent_ord = [ord(x) for x in sent_orig]
    
    # `dp`用来储存不同位置每种标注（B/I）的最大概率值
    dp = np.zeros((2, len(sent_ord)), dtype=float)
    
    # `path`用来储存最大概率对应的上步B/I选择
    #  例如 path[1][7] == 1 意味着第8个（从1开始计数）字符标注I对应的最大概率，其前一步的隐状态为1（I）
    #  例如 path[0][5] == 1 意味着第6个字符标注B对应的最大概率，其前一步的隐状态为1（I）
    #  例如 path[1][1] == 0 意味着第2个字符标注I对应的最大概率，其前一步的隐状态为0（B）
    path = np.zeros((2, len(sent_ord)), dtype=int)
    
    #  TODO: 第一个位置的最大概率值计算（1分）
    dp[0][0] = start_prob[0] * emission_mat[0][sent_ord[0]]
    dp[1][0] = start_prob[1] * emission_mat[1][sent_ord[0]]
    if (dp[0][0] > dp[1][0]):
        path[0][0] = 1
    else:
        path[1][0] = 1
    
    #  TODO: 其余位置的最大概率值计算（填充dp和path矩阵）（2分）
    for i in range(1, len(sent_ord)):
        # 通过比较第i-1个词状态为0与状态为1时计算得到的第i个词状态为0时的最大概率值大小，更新dp和path矩阵在第i个词上的值
        if (dp[0][i-1] * trans_mat[0][0] * emission_mat[0][sent_ord[i]] > dp[1][i-1] * trans_mat[1][0] * emission_mat[0][sent_ord[i]]):
            dp[0][i] = dp[0][i-1] * trans_mat[0][0] * emission_mat[0][sent_ord[i]]
            path[0][i] = 0
        else:
            dp[0][i] = dp[1][i-1] * trans_mat[1][0] * emission_mat[0][sent_ord[i]]
            path[0][i] = 1
        
        if (dp[0][i-1] * trans_mat[0][1] * emission_mat[1][sent_ord[i]] > dp[1][i-1] * trans_mat[1][1] * emission_mat[1][sent_ord[i]]):
            dp[1][i] = dp[0][i-1] * trans_mat[0][1] * emission_mat[1][sent_ord[i]]
            path[1][i] = 0
        else:
            dp[1][i] = dp[1][i-1] * trans_mat[1][1] * emission_mat[1][sent_ord[i]]
            path[1][i] = 1
        
    
    #  `labels`用来储存每个位置最有可能的隐状态
    labels = [0 for _ in range(len(sent_ord))]
    
    #  TODO：计算labels每个位置上的值（填充labels矩阵）（1分）
    for i in range(len(labels)):
        if (dp[0][i] > dp[1][i]):
            labels[i] = 0
        else:
            labels[i] = 1
    
    #  根据lalels生成切分好的字符串
    sent_split = []
    for idx, label in enumerate(labels):
        if label == 1:
            sent_split += [sent_ord[idx], ord("/")]
        else:
            sent_split += [sent_ord[idx]]
    sent_split_str = "".join([chr(x) for x in sent_split])

    return sent_split_str
            

In [24]:
print("viterbi算法分词结果：", viterbi(input_sentence, start_probability, trans_matrix, emission_matrix))

viterbi算法分词结果： 柳纪/宇是/一名/优秀/的/学生/


实现前向算法，计算该句子的概率值

前向算法实现过程：

TODO1： 将初始位置概率值定义为初始概率与发射概率的乘积。

TODO2:  依次更新位置i（从1到len(sent_ord)-1的整数）的最大概率矩阵值。更新位置i状态h的最大概率值（dp[h][i]）的方法为遍历上个位置的所有状态，将所有状态(k)对应的最大概率值（dp[k][i-1]）乘以对应的转移概率（trans_mat[k][h]）再乘以相应的发射概率（emission_mat[h][sent_ord[i]]）进行累加并将结果赋给dp[h][i]。


In [25]:
def compute_prob_by_forward(sent_orig, start_prob, trans_mat, emission_mat):
    """
    前向算法，计算输入中文句子的概率值

    Args:
        sent_orig: str - 输入的句子
        start_prob: numpy.ndarray - 起始概率矩阵
        trans_mat: numpy.ndarray - 转移概率矩阵
        emission_mat: numpy.ndarray - 发射概率矩阵

    Return:
        float - 概率值
    """
    
    #  将汉字转为数字表示
    sent_ord = [ord(x) for x in sent_orig]

    # `dp`用来储存不同位置每种隐状态（B/I）下，到该位置为止的句子的概率
    dp = np.zeros((2, len(sent_ord)), dtype=float)

    # TODO: 初始位置概率的计算（1分）
    dp[0][0] = start_prob[0] * emission_mat[0][sent_ord[0]]
    dp[1][0] = start_prob[1] * emission_mat[1][sent_ord[0]]
    
    # TODO: 先计算其余位置的概率（填充dp矩阵），然后return概率值（1分）
    for i in range(1, len(sent_ord)):
        dp[0][i] = dp[0][i-1] * trans_mat[0][0] * emission_mat[0][sent_ord[i]] + dp[1][i-1] * trans_mat[1][0] * emission_mat[0][sent_ord[i]]
        dp[1][i] = dp[0][i-1] * trans_mat[0][1] * emission_mat[1][sent_ord[i]] + dp[1][i-1] * trans_mat[1][1] * emission_mat[1][sent_ord[i]]

    return sum([dp[i][len(sent_ord)-1] for i in range(2)])

实现后向算法，计算该句子的概率值

后向算法实现流程：

TODO1:  将终末位置的概率初始化为1.（这是因为终末位置一定为终止符<\/s>）

TODO2:  依次更新位置i（从len(sent_ord)-2到0的整数）的最大概率矩阵值。更新位置i状态h的最大概率值（dp[h][i]）的方法为遍历下个位置的所有状态，将所有状态(k)对应的最大概率值（dp[k][i+1]）乘以对应的转移概率（trans_mat[h][k]）再乘以相应的发射概率（emission_mat[k][sent_ord[i+1]]）进行累加并将结果赋给dp[h][i]。


In [26]:
def compute_prob_by_backward(sent_orig, start_prob, trans_mat, emission_mat):
    """
    后向算法，计算输入中文句子的概率值

    Args:
        sent_orig: str - 输入的句子
        start_prob: numpy.ndarray - 起始概率矩阵
        trans_mat: numpy.ndarray - 转移概率矩阵
        emission_mat: numpy.ndarray - 发射概率矩阵

    Return:
        float - 概率值
    """
    
    #  将汉字转为数字表示
    sent_ord = [ord(x) for x in sent_orig]

    # `dp`用来储存不同位置每种隐状态（B/I）下，从结尾到该位置为止的句子的概率
    dp = np.zeros((2, len(sent_ord)), dtype=float)

    # TODO: 终末位置概率的初始化（1分）
    dp[0][len(sent_ord)-1] = 1
    dp[1][len(sent_ord)-1] = 1
    
    # TODO: 先计算其余位置的概率（填充dp矩阵），然后return概率值（1分）
    for i in reversed(range(len(sent_ord)-1)):
        
        dp[0][i] = dp[0][i+1] * trans_mat[0][0] * emission_mat[0][sent_ord[i+1]] + dp[1][i+1] * trans_mat[0][1] * emission_mat[1][sent_ord[i+1]]
        dp[1][i] = dp[0][i+1] * trans_mat[1][0] * emission_mat[0][sent_ord[i+1]] + dp[1][i+1] * trans_mat[1][1] * emission_mat[1][sent_ord[i+1]]

    return sum([dp[i][0] * start_prob[i] * emission_mat[i][sent_ord[0]] for i in range(2)])

In [27]:
print("前向算法概率：", compute_prob_by_forward(input_sentence, start_probability, trans_matrix, emission_matrix))
print("后向算法概率：", compute_prob_by_backward(input_sentence, start_probability, trans_matrix, emission_matrix))

前向算法概率： 4.2486728942315935e-34
后向算法概率： 4.248672894231593e-34


## 任务二：BPE算法用于英文分词

任务二评分标准：

1. 共有7处TODO需要填写，每个TODO计1-2分，共9分，预计代码量50行；
2. 允许自行修改、编写代码完成，对于该情况，请补充注释以便于评分，否则结果不正确将导致较多的扣分；
3. 实验报告(python)/用于说明实验的文字块(jupyter notebook)不额外计分，但不写会导致扣分。

构建空格分词器，将语料中的句子以空格切分成单词，然后将单词拆分成字母加`</w>`的形式。例如`apple`将变为`a p p l e </w>`。

In [28]:
import re
import functools

In [29]:
_splitor_pattern = re.compile(r"[^a-zA-Z']+|(?=')")
_digit_pattern = re.compile(r"\d+")

def white_space_tokenize(corpus):
    """
    先正则化（字母转小写、数字转为N、除去标点符号），然后以空格分词语料中的句子，例如：
    输入 corpus=["I am happy.", "I have 10 apples!"]，
    得到 [["i", "am", "happy"], ["i", "have", "N", "apples"]]

    Args:
        corpus: List[str] - 待处理的语料

    Return:
        List[List[str]] - 二维List，内部的List由每个句子的单词str构成
    """

    tokeneds = [list(
        filter(lambda tkn: len(tkn)>0, _splitor_pattern.split(_digit_pattern.sub("N", stc.lower())))) for stc in corpus
    ]
    
    return tokeneds

编写相应函数构建BPE算法需要用到的初始状态词典

初始状态词典的实现方法为遍历tokenized_corpus中的每一个词列表，将单词拆分为字母间以空格隔开且结尾带有<\/w>的字符串。若字典中已存在该key值，则将其对应的value值加1；若字典中不存在该key值，则将该key值加入词典并将其value设为1。

In [30]:
def build_bpe_vocab(corpus):
    """
    将语料进行white_space_tokenize处理后，将单词每个字母以空格隔开、结尾加上</w>后，构建带频数的字典，例如：
    输入 corpus=["I am happy.", "I have 10 apples!"]，
    得到
    {
        'i </w>': 2,
        'a m </w>': 1,
        'h a p p y </w>': 1,
        'h a v e </w>': 1,
        'N </w>': 1,
        'a p p l e s </w>': 1
     }

    Args:
        corpus: List[str] - 待处理的语料

    Return:
        Dict[str, int] - "单词分词状态->频数"的词典
    """

    tokenized_corpus = white_space_tokenize(corpus)

    bpe_vocab = dict()
    
    # TODO: 完成函数体（1分）
    for sentence in tokenized_corpus:
        for word in sentence:
            list_alpha = list(word)
            split_word = ''
            for alpha in list_alpha:
                split_word += alpha
                split_word += ' '
            split_word += '</w>'

            if split_word in bpe_vocab:
                bpe_vocab[split_word] += 1
            else:
                bpe_vocab[split_word] = 1
            
    return bpe_vocab

编写所需的其他函数

bigram字典的实现方法：

提取bpe_vocab中的keys和values，将它们转换成list类型，接着以空格为分隔符对list中的每个词进行split操作，遍历得到的列表，将两个字母组成一个元组。若字典中已存在该key值，则将其对应的value加上该词对应的频率值；若不存在该key值，则将该key值加入词典并将对应value设置成该词对应的频率。



In [31]:
def get_bigram_freq(bpe_vocab):
    """
    统计"单词分词状态->频数"的词典中，各bigram的频次（假设该词典中，各个unigram以空格间隔），例如：
    输入 bpe_vocab=
    {
        'i </w>': 2,
        'a m </w>': 1,
        'h a p p y </w>': 1,
        'h a v e </w>': 1,
        'N </w>': 1,
        'a p p l e s </w>': 1
    }
    得到
    {
        ('i', '</w>'): 2,
        ('a', 'm'): 1,
        ('m', '</w>'): 1,
        ('h', 'a'): 2,
        ('a', 'p'): 2,
        ('p', 'p'): 2,
        ('p', 'y'): 1,
        ('y', '</w>'): 1,
        ('a', 'v'): 1,
        ('v', 'e'): 1,
        ('e', '</w>'): 1,
        ('N', '</w>'): 1,
        ('p', 'l'): 1,
        ('l', 'e'): 1,
        ('e', 's'): 1,
        ('s', '</w>'): 1
    }

    Args:
        bpe_vocab: Dict[str, int] - "单词分词状态->频数"的词典

    Return:
        Dict[Tuple(str, str), int] - "bigram->频数"的词典
    """

    bigram_freq = dict()
    
    # TODO: 完成函数体（1分）
    words = list(bpe_vocab.keys())
    frequencies = list(bpe_vocab.values())
    for i in range(len(words)):
        alpha_list = words[i].split(' ')
        for alpha_i in range(len(alpha_list)-1):
            alpha_tuple = (alpha_list[alpha_i], alpha_list[alpha_i+1])
            if alpha_tuple in bigram_freq:
                bigram_freq[alpha_tuple] += frequencies[i]
            else:
                bigram_freq[alpha_tuple] = frequencies[i]
    return bigram_freq

合并指定的bigram的实现方法：

提取old_bpe_vocab中的keys和values，将它们转换成list类型，以空格为分隔符对list中的每个词进行split操作，遍历得到的列表。若列表中的连续两个元素正好是bigram中的前后两个元素，则生成的新词的过程中去掉中间的空格；否则依照原样将字母和空格写入新词。将得到的新词列表和原先的频率重新组合成字典类型输出。

In [32]:
def refresh_bpe_vocab_by_merging_bigram(bigram, old_bpe_vocab):
    """
    在"单词分词状态->频数"的词典中，合并指定的bigram（即去掉对应的相邻unigram之间的空格），最后返回新的词典，例如：
    输入 bigram=('i', '</w>')，old_bpe_vocab=
    {
        'i </w>': 2,
        'a m </w>': 1,
        'h a p p y </w>': 1,
        'h a v e </w>': 1,
        'N </w>': 1,
        'a p p l e s </w>': 1
    }
    得到
    {
        'i</w>': 2,
        'a m </w>': 1,
        'h a p p y </w>': 1,
        'h a v e </w>': 1,
        'N </w>': 1,
        'a p p l e s </w>': 1
    }

    Args:
        old_bpe_vocab: Dict[str, int] - 初始"单词分词状态->频数"的词典

    Return:
        Dict[str, int] - 合并后的"单词分词状态->频数"的词典
    """
    
    new_bpe_vocab = dict()

    # TODO: 完成函数体（1分）
    words = list(old_bpe_vocab.keys())
    frequencies = list(old_bpe_vocab.values())
    for i in range(len(words)):
        oldstr = bigram[0] + ' ' + bigram[1]
        newstr = bigram[0] + bigram[1]
        if oldstr in words[i]:
            words[i] = words[i].replace(oldstr, newstr)


    new_bpe_vocab = dict(map(lambda x,y:[x,y], words, frequencies))
    
    
    return new_bpe_vocab

BPE分词列表的实现方法：

首先创建一个新的字典（bpe_dic），用与之前相同的方式拆分bpe_vocab的keys和values，遍历keys中的每个词，将分词对应的频率添加到字典中。将字典转换为元素为元组的列表类型，以元组第0个元素的长度为key值进行排序（若该分词中含有'<\/w>'，由于其只算做一个长度，故要在原有长度的基础上减三），返回得到的结果。

In [33]:
def get_bpe_tokens(bpe_vocab):
    """
    根据"单词分词状态->频数"的词典，返回所得到的BPE分词列表，并将该列表按照分词长度降序排序返回，例如：
    输入 bpe_vocab=
    {
        'i</w>': 2,
        'a m </w>': 1,
        'ha pp y </w>': 1,
        'ha v e </w>': 1,
        'N </w>': 1,
        'a pp l e s </w>': 1
    }
    得到
    [
        ('i</w>', 2),
        ('ha', 2),
        ('pp', 2),
        ('a', 2),
        ('m', 1),
        ('</w>', 5),
        ('y', 1),
        ('v', 1),
        ('e', 2),
        ('N', 1),
        ('l', 1),
        ('s', 1)
     ]

    Args:
        bpe_vocab: Dict[str, int] - "单词分词状态->频数"的词典

    Return:
        List[Tuple(str, int)] - BPE分词和对应频数组成的List
    """
    
    # TODO: 完成函数体（2分）
    bpe_tokens = []
    bpe_dic = {}
    words = list(bpe_vocab.keys())
    frequencies = list(bpe_vocab.values())


    for i in range(len(words)):
        alpha_list = words[i].split(' ')
        for alpha in alpha_list:
            if alpha in bpe_dic:
                bpe_dic[alpha] += frequencies[i]
            else:
                bpe_dic[alpha] = frequencies[i]
    
    

    bpe_tokens = list(bpe_dic.items())
    bpe_tokens = sorted(bpe_tokens, key= lambda s:len(s[0])-3 if '</w>' in s[0] else len(s[0]), reverse=True)

    

    return bpe_tokens

对所给单词进行BPE分词的实现方法：

首先对输入单词的长度进行判别，若长度为0则返回空字符。接着从列表中寻找BPE分词进行子串匹配，若成功匹配，则递归地匹配该字符左边和右边的剩余部分，将二者与匹配子串相加并返回。若最终没能找到匹配的分词，则返回'\<unknown>'。

In [34]:
def print_bpe_tokenize(word, bpe_tokens):
    """
    根据按长度降序的BPE分词列表，将所给单词进行BPE分词，最后打印结果。
    
    思想是，对于一个待BPE分词的单词，按照长度顺序从列表中寻找BPE分词进行子串匹配，
    若成功匹配，则对该子串左右的剩余部分递归地进行下一轮匹配，直到剩余部分长度为0，
    或者剩余部分无法匹配（该部分整体由"<unknown>"代替）
    
    例1：
    输入 word="supermarket", bpe_tokens=[
        ("su", 20),
        ("are", 10),
        ("per", 30),
    ]
    最终打印 "su per <unknown>"

    例2：
    输入 word="shanghai", bpe_tokens=[
        ("hai", 1),
        ("sh", 1),
        ("an", 1),
        ("</w>", 1),
        ("g", 1)
    ]
    最终打印 "sh an g hai </w>"

    Args:
        word: str - 待分词的单词str
        bpe_tokens: List[Tuple(str, int)] - BPE分词和对应频数组成的List
    """
    
    # TODO: 请尝试使用递归函数定义该分词过程（2分）

    def bpe_tokenize(sub_word):
        find = False
        for token in bpe_tokens:
            if len(sub_word) == 0:
                return ''
            if token[0] in sub_word:
                word = sub_word.split(token[0])
                left_word = word[0]
                right_word = word[1]
                find = True
                return bpe_tokenize(left_word) + token[0] + ' ' + bpe_tokenize(right_word)
        if find == False:
            return '<unknown>'
            


    res = bpe_tokenize(word+"</w>")
    print(res)

开始读取数据集并训练BPE分词器

In [35]:
with open("data/news.2007.en.shuffled.deduped.train", encoding="utf-8") as f:
    training_corpus = list(map(lambda l: l.strip(), f.readlines()[:1000]))

print("Loaded training corpus.")

Loaded training corpus.


训练循环的实现过程为：

首先通过调用get_bigram_freq函数获取词表中每个bigram的频次，接着取其中频次最大的一个bigram，最后调用refresh_bpe_vocab_by_merging_bigram函数进行分词合并。迭 代达到training_iter_num时停止训练。

In [45]:
training_iter_num = 300

training_bpe_vocab = build_bpe_vocab(training_corpus)
for i in range(training_iter_num):
    # TODO: 完成训练循环内的代码逻辑（2分）
    bigram_freq = get_bigram_freq(training_bpe_vocab)
    top_bigram = max(bigram_freq.items(), key=lambda d:d[1])[0]
    
    training_bpe_vocab = refresh_bpe_vocab_by_merging_bigram(top_bigram, training_bpe_vocab)



training_bpe_tokens = get_bpe_tokens(training_bpe_vocab)

测试BPE分词器的分词效果

In [46]:
test_word = "naturallanguageprocessing"

print("naturallanguageprocessing 的分词结果为：")
print_bpe_tokenize(test_word, training_bpe_tokens)

naturallanguageprocessing 的分词结果为：
n atur al lan gu age pro ce s sing</w> 
